In [90]:
TRAINING_DATA_PATH = "/Users/ahshafi/Documents/NLP/testing/Transaction_Naration_Data_Set.csv"
import pandas as pd
data = pd.read_csv(TRAINING_DATA_PATH)
data

,NO,AC_ID,AMOUNT,DOC_DATE,NARATION,DR_CR
0,0,3644877,342800.0,01-NOV-22,Cash Withdraw,Debit
1,1,1175256,342800.0,01-NOV-22,Cash Withdrawal From A/C No.:,Credit
2,2,2298226,1500.0,01-NOV-22,"Cash withdraw from Micro Merchant point, MM A/C",Debit
3,3,4066427,1500.0,01-NOV-22,Credit Against Merchant Cash Withdraw Service ...,Credit
4,4,1978564,2550.0,01-NOV-22,"Cash withdraw from Micro Merchant point, MM A/C",Debit
5,5,4890311,2550.0,01-NOV-22,Credit Against Merchant Cash Withdraw Service ...,Credit
6,6,38418,500.0,01-NOV-22,Cash Deposit to A/C No.:,Debit
7,7,5833368,500.0,01-NOV-22,Cash Deposit,Credit
8,8,742629,500.0,01-NOV-22,Cash Deposit to A/C No.:,Debit
9,9,3683842,500.0,01-NOV-22,Cash Deposit,Credit


In [91]:
data.columns

Index(['NO', 'AC_ID', 'AMOUNT', 'DOC_DATE', 'NARATION', 'DR_CR'], dtype='object')

In [92]:
narration = data['NARATION'].to_frame()
narration

,NARATION
0,Cash Withdraw
1,Cash Withdrawal From A/C No.:
2,"Cash withdraw from Micro Merchant point, MM A/C"
3,Credit Against Merchant Cash Withdraw Service ...
4,"Cash withdraw from Micro Merchant point, MM A/C"
5,Credit Against Merchant Cash Withdraw Service ...
6,Cash Deposit to A/C No.:
7,Cash Deposit
8,Cash Deposit to A/C No.:
9,Cash Deposit


In [93]:
SAMP_CNT = 4
narration.sample(n=SAMP_CNT)

,NARATION
3,Credit Against Merchant Cash Withdraw Service ...
11,Credit Against Merchant Cash Withdraw Service ...
20,Agri agent commission of a/c
5,Credit Against Merchant Cash Withdraw Service ...


In [94]:
no_dup = narration.drop_duplicates()
df = no_dup.copy(deep = True)
df.rename(columns = {'NARATION' : 'text'}, inplace = True)
df

,text
0,Cash Withdraw
1,Cash Withdrawal From A/C No.:
2,"Cash withdraw from Micro Merchant point, MM A/C"
3,Credit Against Merchant Cash Withdraw Service ...
6,Cash Deposit to A/C No.:
7,Cash Deposit
12,Debit For New ATM Card Charge.
19,Debit Against Palli Bidyut Bill Collection.
20,Agri agent commission of a/c


In [95]:
def toLower(df):
  df['text'] = df['text'].apply(lambda z : str(z).lower())

def eliminateNonAlphabet(df):
  df['text'] = df['text'].str.replace(r'[^\w\s]+', ' ')
  df['text'] = df['text'].replace( r'[^A-Za-z ]+', '', regex=True)

def tokenize(df):
  import nltk
  nltk.download('punkt')
  df['text'] = df['text'].apply(nltk.word_tokenize)

def dropEmpty(df):
  df.drop(df[df['text'].map(len) == 0].index, inplace = True)

def lematize(df):
  import nltk
  from nltk.stem import WordNetLemmatizer
  nltk.download('wordnet')
  nltk.download('omw-1.4')
  lemmatizer = WordNetLemmatizer()
  df['text'] = df['text'].apply(lambda z : [lemmatizer.lemmatize(x) for x in z])

def removeWord(df, stop):
  df['text'] = df['text'].apply(lambda z : [word for word in z if word not in stop])

def removeGeneralStopWords(df):
  import nltk
  from nltk.corpus import stopwords
  nltk.download("stopwords")
  stop = set(stopwords.words("english"))
  removeWord(df, stop)

def showAppearances(df1, df2, text):
  return df1.loc[df2[df2['text'].map(lambda z : text in z) == True].index]

def removeDup(df):
  df.drop_duplicates(inplace = True)

# def removeEmptyTokensRows(df):
#   df.drop(df[df['text'].map(len) == 0].index, inplace = True)

def removeCustomWords(df):
  stops = ['ac', 'ae', 'md', 'akter', 'hossain', 'rahman', 'ltd', 'bazar', 'unnayan', 'dol', 'polli', 'mohila', 'ad', 'mia', 'ali', 'uddin', 
         'begum', 'tk', 'islam', 'amt', 'abdul', 'hasan', 'sonali', 'alam', 'miah', 'bangladesh' , 'per', 'taka',
         'da', 'vai', 'agrani' , 'ahmed', 'hossen', 'khan', 'abu', 'janata', 'al', 'unnayon', 'b', 'amin', 'mamun', 'khatun', 'unnoyon',  'nur', 'haque']
  notSure = ['mo', 'cap','fvg', 'islami', 'br', 'id' , 'dal', 'purush', 'nid', 'purpose', 'title', 'para', 'roll']
  months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
  removeWord(df, set(stops + notSure + months))

def replaceWord(df):
  def func(lst):
    try:
      index = lst.index('withdrawal')
      lst[index] = 'withdraw'
    except:
      None
    return lst
  df['text'] = df['text'].apply(func)

def removeNonEnglishWord(df,keep):
  import nltk
  nltk.download('words')
  from nltk.corpus import words
  wordSet = set(words.words())
  def func(row):
    r2 = []
    for w in row:
      if (len(str(w)) > 2) and ((w in wordSet) or (w in keep)):
        r2.append(w)

    return r2 

  df['text'] = df['text'].apply(func)


In [96]:
toLower(df)
df

,text
0,cash withdraw
1,cash withdrawal from a/c no.:
2,"cash withdraw from micro merchant point, mm a/c"
3,credit against merchant cash withdraw service ...
6,cash deposit to a/c no.:
7,cash deposit
12,debit for new atm card charge.
19,debit against palli bidyut bill collection.
20,agri agent commission of a/c


In [97]:
eliminateNonAlphabet(df)
df

/var/folders/gs/zgk9p52104g9k3frjpgwl9tw0000gn/T/ipykernel_30048/4139995585.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'[^\w\s]+', ' ')


,text
0,cash withdraw
1,cash withdrawal from a c no
2,cash withdraw from micro merchant point mm a c
3,credit against merchant cash withdraw service ...
6,cash deposit to a c no
7,cash deposit
12,debit for new atm card charge
19,debit against palli bidyut bill collection
20,agri agent commission of a c


In [98]:
tokenize(df)
df

[nltk_data] Downloading package punkt to /Users/ahshafi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text
0,"[cash, withdraw]"
1,"[cash, withdrawal, from, a, c, no]"
2,"[cash, withdraw, from, micro, merchant, point,..."
3,"[credit, against, merchant, cash, withdraw, se..."
6,"[cash, deposit, to, a, c, no]"
7,"[cash, deposit]"
12,"[debit, for, new, atm, card, charge]"
19,"[debit, against, palli, bidyut, bill, collection]"
20,"[agri, agent, commission, of, a, c]"


In [99]:
lematize(df)
df

[nltk_data] Downloading package wordnet to /Users/ahshafi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ahshafi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text
0,"[cash, withdraw]"
1,"[cash, withdrawal, from, a, c, no]"
2,"[cash, withdraw, from, micro, merchant, point,..."
3,"[credit, against, merchant, cash, withdraw, se..."
6,"[cash, deposit, to, a, c, no]"
7,"[cash, deposit]"
12,"[debit, for, new, atm, card, charge]"
19,"[debit, against, palli, bidyut, bill, collection]"
20,"[agri, agent, commission, of, a, c]"


In [100]:
removeGeneralStopWords(df)
df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahshafi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text
0,"[cash, withdraw]"
1,"[cash, withdrawal, c]"
2,"[cash, withdraw, micro, merchant, point, mm, c]"
3,"[credit, merchant, cash, withdraw, service, c]"
6,"[cash, deposit, c]"
7,"[cash, deposit]"
12,"[debit, new, atm, card, charge]"
19,"[debit, palli, bidyut, bill, collection]"
20,"[agri, agent, commission, c]"


In [101]:
removeCustomWords(df)
df

,text
0,"[cash, withdraw]"
1,"[cash, withdrawal, c]"
2,"[cash, withdraw, micro, merchant, point, mm, c]"
3,"[credit, merchant, cash, withdraw, service, c]"
6,"[cash, deposit, c]"
7,"[cash, deposit]"
12,"[debit, new, atm, card, charge]"
19,"[debit, palli, bidyut, bill, collection]"
20,"[agri, agent, commission, c]"


In [102]:
replaceWord(df)
df

,text
0,"[cash, withdraw]"
1,"[cash, withdraw, c]"
2,"[cash, withdraw, micro, merchant, point, mm, c]"
3,"[credit, merchant, cash, withdraw, service, c]"
6,"[cash, deposit, c]"
7,"[cash, deposit]"
12,"[debit, new, atm, card, charge]"
19,"[debit, palli, bidyut, bill, collection]"
20,"[agri, agent, commission, c]"


In [103]:
removeNonEnglishWord(df, set(['eftn', 'ft', 'bkash', 'nogod', 'rtgs', 'pos', 'sme', 'mst']))
df

[nltk_data] Downloading package words to /Users/ahshafi/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


,text
0,"[cash, withdraw]"
1,"[cash, withdraw]"
2,"[cash, withdraw, micro, merchant, point]"
3,"[credit, merchant, cash, withdraw, service]"
6,"[cash, deposit]"
7,"[cash, deposit]"
12,"[debit, new, card, charge]"
19,"[debit, palli, bill, collection]"
20,"[agent, commission]"


In [104]:
dropEmpty(df)
df

,text
0,"[cash, withdraw]"
1,"[cash, withdraw]"
2,"[cash, withdraw, micro, merchant, point]"
3,"[credit, merchant, cash, withdraw, service]"
6,"[cash, deposit]"
7,"[cash, deposit]"
12,"[debit, new, card, charge]"
19,"[debit, palli, bill, collection]"
20,"[agent, commission]"


In [105]:
df.drop_duplicates()

,text
0,"[cash, withdraw]"
2,"[cash, withdraw, micro, merchant, point]"
3,"[credit, merchant, cash, withdraw, service]"
6,"[cash, deposit]"
12,"[debit, new, card, charge]"
19,"[debit, palli, bill, collection]"
20,"[agent, commission]"


In [ ]:
!pip freeze > requirements.txt